<a href="https://colab.research.google.com/github/brmontgomery/483-Intro_To_ML_Assignments/blob/main/CS483_Project1_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project 1 - Anomaly Detection
**Group members:** Zach Hofmeister, Brian Montgomery, and Karla Cabrera. <br>
**Section:** CS 483-02 <br>
**Semester:** Fall 2021 <br>
**Project Summary:** For this project we used Python inside of Jupyter to import and examine the contents of a csv file containing class attendance data. We implimented two methods to check for outliers in the data (Tukey's Fences and standard deviation), which we used to detect students who attended class for very little time. We compared the methods and found that the standard deviation method detected fewer ouliers than the Tukey method did. We gained a lot of experience using Jupyter Notebooks inside of Google Colab, for instance learning how to to connect it to a local instance to use Python 3.8.

#Experiment 1
Use the *csv* module to load the dataset and examine the contents of the first few rows



In [ ]:
import csv

#open the file
csvfile = open('participants.csv', newline='')
reader = csv.DictReader(csvfile)

#list to save each row for later use
readerList = []

#save each row in the readerList and print the row
displayRows = 5
for row in reader:
  readerList.append(row)
  displayRows -= 1
  if displayRows >= 0:
    print(row['Student Name'], row['Week 1'], row['Week 2'], row['Week 3'], row['Week 4'], row['Week 5'])

Adrian Ellison 77 154 4 170 175
Ophelia Mcphee 179 151 164 173 171
Yasir Fenton 180 47 164 168 169
Benny Arias 180 152 161 170 170
Tamara Cottrell 183 79 161 173 168


##Results
We were able to get the dataset to load into a csv.dictReader. Since the dict reader uses an iterator that points to the end after one loop, we decided to move the data into a list for further use. Shown are the first 5 rows of our list of reader csv lines.

-------------------------------------------------------------------------------

#Experiment 2
Load the statistics module and use it to find the mean and median of Week 1’s data.

In [ ]:
import statistics

#create a list for the week 1 data
listW1 = []

#populate week1List
for row in readerList:
  listW1.append(int(row['Week 1']))

#get mean
meanW1 = statistics.mean(listW1)
print('Mean: ', meanW1)

#get median
medianW1 = statistics.median(listW1)
print('Median:', medianW1)


Mean:  161
Median: 175


##Results
Using the previously loaded data and the statistics library, we were able to separate out the data for week 1 and find the mean and median of the week 1 data.

-------------------------------------------------------------------------------

#Experiment 3
Find the quartiles for Week 1.


In [ ]:
#get quantiles
quantW1 = statistics.quantiles(listW1, n=4)
print(quantW1)

[174.0, 175.0, 179.0]


##Results
Using the statistics library, we were able to find the quantiles of the week 1 data. This is essentially the same math as finding the median, however we also get the median of the first half of the ordered data and the median of the second half of the ordered data. These are the basis for our Tukey values in the next experiment.

-------------------------------------------------------------------------------


#Experiment 4
In order to record attendance, we want to find the students who logged into the Zoom meeting but did not attend the entire lecture. In order to do this, we can look for outliers in the data
Tukey’s fences are a simple method to define outliers in terms of the interquartile range. (In fact, they are usually included as whiskers in box plots in order to visualize outliers).
Use this method with k = 1.5 to find the outliers in the Week 1 attendance data

In [ ]:
#getting our tukey values
k = 1.5
interQRange = k * (quantW1[2] - quantW1[0])
tukeyLowerW1 = quantW1[0] - interQRange
tukeyUpperW1 = quantW1[2] + interQRange
#print(tukeyLowerW1, tukeyUpperW1);

#get outliers from week 1 data
tukeyOutliersW1 = []
for row in readerList:
  if float(row['Week 1']) < tukeyLowerW1 or float(row['Week 1']) > tukeyUpperW1:
    tukeyOutliersW1.append({ row['Student Name']: row['Week 1']})
print('Tukey Outliers:', tukeyOutliersW1)

Tukey Outliers: [{'Adrian Ellison': '77'}, {'Tayla Sparrow': '51'}, {'Owain Emerson': '9'}, {'Alaya Dickinson': '24'}]


##Results
For experiment 4, we used a simple method called Tukey fences in order to look for the outliers using k=1.5 value. Using a for loop, we were able to compare each row to identify if it was an outlier. If it was an outlier, it would appended into a list.

-------------------------------------------------------------------------------

#Experiment 5
Recall that in a normal distribution, 99.7% of the values lie within three standard deviations from the mean. If we assume that our data are normally distributed, this gives us another way to find outliers.
Compute the standard deviation for the Week 1 attendance data, then use this method to find the outliers. Do your results agree with experiment (4)?


In [ ]:
#get std deviation of week 1
sdW1 = statistics.pstdev(listW1)
#print('Standard Deviation: ', sdW1)

#get the upper and lower bounds
sdUpperW1 = meanW1 + 3 * sdW1
sdLowerW1 = meanW1 - 3 * sdW1

#get the outliers by std deviation
sdOutliersW1 = []
for row in readerList:
  if float(row['Week 1']) < sdLowerW1 or float(row['Week 1']) > sdUpperW1:
    sdOutliersW1.append({ row['Student Name']: row['Week 1']})
print('Std. Deviation Outliers:', sdOutliersW1)

Std. Deviation Outliers: [{'Owain Emerson': '9'}, {'Alaya Dickinson': '24'}]


##Results
In experiment 5, we used the alternative method of standard deviations to find outliers in the data. We noticed that this method finds a smaller range of outliers in this dataset than the Tukey fences. This could be useful depending on what we are trying to find.
In this case, should the trend of finding a smaller range of outliers hold true, we could use the standard deviation method to find students who only showed up for roll-call, had internet disruptions early on, or only tuned into the end of the class session. We will only know once we compare the results of more datasets.

-------------------------------------------------------------------------------


#Experiment 6
Define a function tardy_iqr() to make experiment (4) repeatable. This function should take the name of a column (e.g. 'Week 1') and return a list of names for whom the number of minutes is below the lower Tukey fence (e.g. ['Alaya Dickinson', 'Owain Emerson']). Verify that this function returns the same results as experiment (4).


In [ ]:
#function to get all outliers below the lower tukey for a given week
def tardy_iqr( name ):
    times = []

    #populate list for the week
    for row in readerList:
        times.append(int(row[name]))

    #get quantiles
    quant = statistics.quantiles(times, n=4)

    #getting our tukey values
    k = 1.5
    interQRange = k * (quant[2] - quant[0])
    tukeyLower = quant[0] - interQRange

    #get outliers from that week's data
    tukeyOutliers = []
    for row in readerList:
        if float(row[name]) < tukeyLower:
            tukeyOutliers.append(row['Student Name'])
    return tukeyOutliers

In [ ]:
print('Tukey Outliers for Week 1:', tardy_iqr('Week 1'))

Tukey Outliers for Week 1: ['Adrian Ellison', 'Tayla Sparrow', 'Owain Emerson', 'Alaya Dickinson']


##Results
For experiment 6, we combined the code from experiments 3 and 4 into a function that can be used to find the lower outliers from any week with the Tukey's Fences method. Unlike experiment 4, since we are checking for students with low participation we only test against the lower tukey bound, and we output a list of names only. Below we can see that it gives the same result for Week 1 as for experiment 4.

-------------------------------------------------------------------------------

#Experiment 7
Define a second function, tardy_stdev(), with the same interface as experiment (6) but using the method of experiment (5) and verify that its results match that experiment.

In [ ]:
def tardy_stdev(name):
  #create a list for the column data
  listCol = []

  #populate listCol
  for row in readerList:
    listCol.append(int(row[name]))

  #get mean
  mean = statistics.mean(listCol)
  #print('Mean: ', mean)

  #get std deviation of data
  sd = statistics.pstdev(listCol)
  #print('Standard Deviation: ', sd)

  #get the lower bounds
  sdLower = mean - 3 * sd

  #get the outliers by std deviation
  sdOutliers = []
  for row in readerList:
    if float(row[name]) < sdLower:
      sdOutliers.append(row["Student Name"])
  return sdOutliers

In [ ]:
print('Std. Deviation Outliers for Week 1:', tardy_stdev('Week 1'))

Std. Deviation Outliers for Week 1: ['Owain Emerson', 'Alaya Dickinson']


##Results
Using the method of experiment 5 we were able to define the function tardy_stdev. We first created a for loop that would append the name from the row, get the mean, standard deviation of data, and lower bound. We then created another for loop that will check and compare if the student is an outlier and add their name to a list. After the for loop is completed, it would output the list of student names who are outliers by standard deviation.

-------------------------------------------------------------------------------

#Experiment 8
Compare the results of tardy_iqr() and tardy_stdev() on Weeks 2-5.

In [ ]:
print("Tardy output for weeks 2-5 using Tukey Fences")
print('Tukey Outliers for Week 2:', tardy_iqr('Week 2'))
print('Tukey Outliers for Week 3:', tardy_iqr('Week 3'))
print('Tukey Outliers for Week 4:', tardy_iqr('Week 4'))
print('Tukey Outliers for Week 5:', tardy_iqr('Week 5'))
print("\nTardy output for weeks 2-5 using Standard Deviations")
print('Std. Deviation Outliers for Week 2:', tardy_stdev('Week 2'))
print('Std. Deviation Outliers for Week 3:', tardy_stdev('Week 3'))
print('Std. Deviation Outliers for Week 4:', tardy_stdev('Week 4'))
print('Std. Deviation Outliers for Week 5:', tardy_stdev('Week 5'))

Tardy output for weeks 2-5 using Tukey Fences
Tukey Outliers for Week 2: ['Yasir Fenton', 'Tamara Cottrell', 'Jazmin Foreman', 'Bear Zuniga', 'Miles Lyons']
Tukey Outliers for Week 3: ['Adrian Ellison', 'Adeline Jordan', 'Jaye Sweeney']
Tukey Outliers for Week 4: []
Tukey Outliers for Week 5: ['Jazmin Foreman', 'Sanjay Edwards', 'Alfie-James Pierce', 'Saffa Brook']

Tardy output for weeks 2-5 using Standard Deviations
Std. Deviation Outliers for Week 2: ['Miles Lyons']
Std. Deviation Outliers for Week 3: ['Adrian Ellison']
Std. Deviation Outliers for Week 4: []
Std. Deviation Outliers for Week 5: ['Jazmin Foreman']


##Results
Comparing the two results, it is immediately obvious that the Tukey fences are finding more outliers in the data. As this is consistent in all of the datasets where there are outliers (1-5) we can assume that our observations found in experiment 5 will hold true for a large number of datasets. These are that:
1. We can use the standard deviation method to find students who only showed up for roll-call, had internet disruptions early on, or only tuned into the end of the class session.
2. With the Tukey's however, you maybe able to find students who were in class for a moderate amount of time as well as finding those who were only in class for the minimum number of minutes.

It is possible that changing the number of standard deviations used to find the outliers may change these results.

-------------------------------------------------------------------------------
